<img src="https://github.com/jina-ai/docarray/blob/main/.github/banner.png?raw=true" alt="Create Disco Diffusion artworks in one-line" width="150px">
<p align=center>
<a href="https://pypi.org/project/discoart/"><img src="https://img.shields.io/pypi/v/discoart?style=flat-square&amp;label=Release" alt="PyPI"></a>
<a href="https://slack.jina.ai"><img src="https://img.shields.io/badge/Slack-3.1k-blueviolet?logo=slack&amp;logoColor=white&style=flat-square"></a>
<a href="https://colab.research.google.com/github/jina-ai/dalle-flow/blob/main/client.ipynb"><img src="https://img.shields.io/badge/Open-in%20Colab-brightgreen?logo=google-colab&style=flat-square" alt="Open in Google Colab"/></a>
</p>

😎🪩 **If you are already a DD user**: you are ready to go! There is no extra learning, DiscoArt respects the same parameter semantics as DD5.2. So just unleash your creativity!
- Note that DiscoArt does not support animation generation and `image_prompt` (which was marked as uneffective? in original DD 5.2).

👶🎨 **If you are a [DALLE-Flow](https://github.com/jina-ai/dalle-flow/)/DALLE-Mini user**: you may want to take step by step, as Disco Diffusion works in a very different way than DALLE. It is much more advanced and powerful: e.g. Disco Diffusion can take weighted & structured text prompts; it can initialize from a image with controlled noise; and there are way more parameters one can tweak. Impatient prompt like `"armchair avocado"` will give you nothing but confusion and frustration. I highly recommend you to check out the following resources before trying your own prompt:
- [Zippy's Disco Diffusion Cheatsheet v0.3](https://docs.google.com/document/d/1l8s7uS2dGqjztYSjPpzlmXLjl5PM3IGkRWI3IiCuK7g/mobilebasic)
- [EZ Charts - Diffusion Parameter Studies](https://docs.google.com/document/d/1ORymHm0Te18qKiHnhcdgGp-WSt8ZkLZvow3raiu2DVU/edit#)
- [Disco Diffusion 70+ Artist Studies](https://weirdwonderfulai.art/resources/disco-diffusion-70-plus-artist-studies/)

# Install

In [ ]:
!pip install discoart

# Create artworks

In [ ]:
from discoart import create

da = create()

That's it. It is that simple.

# Specify parameters

In [ ]:
from discoart import create

create(text_prompts='A painting of sea cliffs in a tumultuous storm, Trending on ArtStation.',
       init_image='https://d2vyhzeko0lke5.cloudfront.net/2f4f6dfa5a05e078469ebe57e77b72f0.png',
       skip_steps=100)

# Pull results

At anytime on any machine, you can pull the real-time results (including paramters, intermedidate diffusion steps, final results) with a session ID:

> Please replace `discoart-3205998582` to your own when you run the above 2 cells!

In [ ]:
from docarray import DocumentArray

da = DocumentArray.pull('discoart-3205998582')

## Plot all final results

In [ ]:
da.plot_image_sprites(skip_empty=True, fig_size=(10, 10), show_index=True)

## Plot the final results of run `0`

In [ ]:
da[0].display()



## Plot all intermediate results of the run `0`

In [ ]:
da[0].chunks.plot_image_sprites(skip_empty=True, fig_size=(10, 10), show_index=True)

## Get the parameters of run `0`

In [ ]:
da[0].tags